In [58]:
#import libraries
import pandas as pd
import re
import time
import os
from itertools import chain
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option("display.max_rows", None)
#check working path
os.getcwd()

'/Users/marysolomon/Desktop/KPOPThesis'

In [59]:
#Load and head artist data.
artist_df = pd.read_csv("artist_df.csv",index_col=0).tail(1)
artist_df.head(n = 5)

,Artist,ID,Type,Gender,Generation,DebutYear
180,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,solo,male,1.0,1994


In [60]:
# Load and head songs meta data table
songs_df = pd.read_csv("songs_meta_table_unclean3.csv",index_col=0)
print('total rows in song meta data table: ', len(songs_df))
#only keep unique song uri ids.
print('total unique songs by uri table: ', len(pd.unique(songs_df['song_uri'])))
#songs_df = songs_df[songs_df['song_uri'].drop_duplicates()]
songs_df.head(n=5)

total rows in song meta data table:  159
total unique songs by uri table:  159


,song_name,song_uri,album,album_uri,artist,artist_uri,release_date,popularity,duration_ms
0,Delicious (Your Lips),spotify:track:5CkAYc5Y3leBLK6Wd59It0,Back to Stage,spotify:album:6SmAKP3u529E7LpnjFsSVf,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,12,214786
1,뒹굴뒹굴,spotify:track:0otd8yQz0lUg2lAVKqSWNl,Halftime,spotify:album:5p7fn6NyIiI1fhkh9mQlT0,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2013-09-09,0,203719
2,살아있네 Still Alive,spotify:track:4ERsViP3OTdJ9qIwpsUSTj,Still Alive,spotify:album:5HiT4lbTgn7VKDAX5Lecql,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2016-04-10,23,174953
3,Still Alive,spotify:track:2uDbAd6taMMeoyOAc9leBU,J.Y. Park BEST (Selected Edition),spotify:album:38rFlt6IxeZXsxPb7PcMn5,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2020-10-07,26,177733
4,Rewind,spotify:track:6fHcZvUm9G76hlLG3PjAfL,Sad Freedom,spotify:album:2OCK4oigskg6ZQ9mBm9zii,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2009-12-01,5,235441


In [61]:
# see if number of artists match
#detect artists to remove
uri1 = pd.unique(artist_df['ID'])
uri2 = pd.unique(songs_df['artist_uri'])
artists1 = artist_df['Artist']
artists2 = pd.unique(songs_df['artist'])

print(len(artists1))
print(len(uri1))
print(len(uri2))
print(len(artists2))

1
1
3
3


In [62]:
#### identifying subset of the artist table
# artists1[154:]
# artist_df[154:]

In [63]:
#### Take a look at all the artists included in the unclean songs list.
artists_sorted = songs_df[['artist','artist_uri']].drop_duplicates().sort_values(by=['artist'], ascending=True)
artists_sorted
#### There are clearly alot that we do not want to include unless there was a collab with a kpop artist

,artist,artist_uri
0,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B
109,Kenji Bunch,spotify:artist:44DPjVo7zPKN3Ejvx34xhR
98,Rain,spotify:artist:5L4EafeXwZ0stGuPtGr5Tz


In [64]:
#detect artists to remove based on URI
uri_rm = set(uri2) - set(uri1)
len(uri_rm)

2

In [65]:
#### look at list of artists to remove based on artist name
artist_rm = set(artists2) - set(artists1)
len(artist_rm)
#artist_rm 

2

In [66]:
### detect which artists from my list that are not detected in the songs collection....
artist_om = set(artists1) - set(artists2)
len(artist_om)
artist_om

0

set()

In [67]:
# prepping to clean the songs meta data file.
# removes songs if their artist uris are in the uris to remove (artists not in scope of research)
type(songs_df['artist_uri'].isin(uri_rm)) #pandas boolean
# tilde removes rows where this condition is true.
songs_clean = songs_df[~songs_df['artist_uri'].isin(uri_rm)]
len(songs_clean)
songs_clean['artist'].drop_duplicates().head(n=10)

pandas.core.series.Series

157

0    J.Y. Park
Name: artist, dtype: object

In [68]:
#This code finds song titles that contain artist name.
# songs_df['song_name'].str.contains('|'.join(artists1)).head(n=5)
# type(songs_df['song_name'].str.contains('|'.join(artists1))) #pandas boolean
# songs_df[songs_df['song_name'].str.contains('|'.join(artists1))].head(n=10)
# #this code finds artist names that contain the artist name
# songs_df['artist'].str.contains('|'.join(artists1)).head(n=5)
# type(songs_df['artist'].str.contains('|'.join(artists1)))
# songs_df[songs_df['artist'].str.contains('|'.join(artists1))].head(n=10)

In [69]:
### Put all three together separated by and operators.
#these are exceptions ot uri not matching
# songs_keep = songs_df[(songs_df['artist_uri'].isin(uri_rm)) & 
#                       ((songs_df['song_name'].str.contains('|'.join(artists1))) |
#                       (songs_df['artist'].str.contains('|'.join(artists1))))]
# len(songs_keep)
# songs_keep

In [70]:
#### songs to remove
# songs_remove = songs_df[(songs_df['artist_uri'].isin(uri_rm)) & 
#                       ~((songs_df['song_name'].str.contains('|'.join(artists1))) |
#                       (songs_df['artist'].str.contains('|'.join(artists1))))]
# len(songs_remove)
# songs_remove.head(n=10)

In [71]:
#### STEP 1. DROP IRRELEVANT ARTISTS ####
### drop songs in song meta table if 
### 1. the artist uri from artist dataframe is not present AND
### (2. for artist name is not in the song titles OR
### 3. artist name from artist df is not present in the artist names from songs df.)
#'|'.join(artists1)
#### current logic isn't working? need to execute more successfully.
songs_clean = songs_df[~((songs_df['artist_uri'].isin(uri_rm)) & 
                      ~((songs_df['song_name'].str.contains('|'.join(artists1))) |
                      (songs_df['artist'].str.contains('|'.join(artists1)))))]
print('total songs after unwanted artists removed: ', len(songs_clean))
songs_clean.head(n=10)

total songs after unwanted artists removed:  157


,song_name,song_uri,album,album_uri,artist,artist_uri,release_date,popularity,duration_ms
0,Delicious (Your Lips),spotify:track:5CkAYc5Y3leBLK6Wd59It0,Back to Stage,spotify:album:6SmAKP3u529E7LpnjFsSVf,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,12,214786
1,뒹굴뒹굴,spotify:track:0otd8yQz0lUg2lAVKqSWNl,Halftime,spotify:album:5p7fn6NyIiI1fhkh9mQlT0,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2013-09-09,0,203719
2,살아있네 Still Alive,spotify:track:4ERsViP3OTdJ9qIwpsUSTj,Still Alive,spotify:album:5HiT4lbTgn7VKDAX5Lecql,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2016-04-10,23,174953
3,Still Alive,spotify:track:2uDbAd6taMMeoyOAc9leBU,J.Y. Park BEST (Selected Edition),spotify:album:38rFlt6IxeZXsxPb7PcMn5,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2020-10-07,26,177733
4,Rewind,spotify:track:6fHcZvUm9G76hlLG3PjAfL,Sad Freedom,spotify:album:2OCK4oigskg6ZQ9mBm9zii,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2009-12-01,5,235441
5,방문을 닫으면 (Behind closed doors),spotify:track:6dmNuJysRUqX2CDA4q2MGs,24/34,spotify:album:7cPUxsJL1nMh87bld2LyTo,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2015-04-12,14,215955
6,위험한 장난,spotify:track:0mnP50PPZLrZm7tOTmui7N,Back To Stage,spotify:album:0R52ET7vG1Hy7eGouTxCve,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,5,199613
7,"Fire (feat. Conan O'Brien, Steven Yeun, 박지민 Ji...",spotify:track:0udhVlaFTeXvlmYWaFo66D,Still Alive,spotify:album:4ueWIlgAiwx4JyAHBCXOwt,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2016-04-10,32,185886
8,엇갈렸어 (Feat. T 윤미래),spotify:track:40nGrvrNoQlspovlWh8Acq,Back To Stage,spotify:album:41h8n1l24t2dq0N2e5lsll,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,1,242320
9,Criss Cross,spotify:track:1kXDAC69E08pLYxyFRcLgn,Back to Stage,spotify:album:6SmAKP3u529E7LpnjFsSVf,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,10,242320


In [72]:
#### STEP 2. REMOVE LIVE/TOUR OR COMPILATION ALBUMS ####

words_rm = ['\slive\s', ' tour ', ' concert ', 'special event', 'best of']

#### Cleaning out albums with the following words in them: 'live', 'tour', 'concert', 'special event', 'the best of'
album_names = songs_clean['album'].drop_duplicates()
print('total number of albums: ',len(album_names))
album_names.head(n=10)

### Filter for albums with the keywords to see if successful :) 
# album_rm = songs_df[songs_df['album'].str.lower().str.contains('|'.join(words_rm))]['album'].drop_duplicates()
# print('total number of albums to remove: ',len(album_rm))
# album_rm

### removing songs from these albums from the dataset
songs_clean = songs_clean[~songs_clean['album'].str.lower().str.contains('|'.join(words_rm))]
print('number of songs when live/tour albums are removed: ' , len(songs_clean))
songs_clean.head(n=25)

total number of albums:  21


0                         Back to Stage
1                              Halftime
2                           Still Alive
3     J.Y. Park BEST (Selected Edition)
4                           Sad Freedom
5                                 24/34
6                         Back To Stage
14                                 Game
15      Spring - 5 Songs for a New Love
23                    나는 배우다 Movie Star
Name: album, dtype: object

number of songs when live/tour albums are removed:  157


,song_name,song_uri,album,album_uri,artist,artist_uri,release_date,popularity,duration_ms
0,Delicious (Your Lips),spotify:track:5CkAYc5Y3leBLK6Wd59It0,Back to Stage,spotify:album:6SmAKP3u529E7LpnjFsSVf,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,12,214786
1,뒹굴뒹굴,spotify:track:0otd8yQz0lUg2lAVKqSWNl,Halftime,spotify:album:5p7fn6NyIiI1fhkh9mQlT0,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2013-09-09,0,203719
2,살아있네 Still Alive,spotify:track:4ERsViP3OTdJ9qIwpsUSTj,Still Alive,spotify:album:5HiT4lbTgn7VKDAX5Lecql,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2016-04-10,23,174953
3,Still Alive,spotify:track:2uDbAd6taMMeoyOAc9leBU,J.Y. Park BEST (Selected Edition),spotify:album:38rFlt6IxeZXsxPb7PcMn5,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2020-10-07,26,177733
4,Rewind,spotify:track:6fHcZvUm9G76hlLG3PjAfL,Sad Freedom,spotify:album:2OCK4oigskg6ZQ9mBm9zii,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2009-12-01,5,235441
5,방문을 닫으면 (Behind closed doors),spotify:track:6dmNuJysRUqX2CDA4q2MGs,24/34,spotify:album:7cPUxsJL1nMh87bld2LyTo,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2015-04-12,14,215955
6,위험한 장난,spotify:track:0mnP50PPZLrZm7tOTmui7N,Back To Stage,spotify:album:0R52ET7vG1Hy7eGouTxCve,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,5,199613
7,"Fire (feat. Conan O'Brien, Steven Yeun, 박지민 Ji...",spotify:track:0udhVlaFTeXvlmYWaFo66D,Still Alive,spotify:album:4ueWIlgAiwx4JyAHBCXOwt,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2016-04-10,32,185886
8,엇갈렸어 (Feat. T 윤미래),spotify:track:40nGrvrNoQlspovlWh8Acq,Back To Stage,spotify:album:41h8n1l24t2dq0N2e5lsll,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,1,242320
9,Criss Cross,spotify:track:1kXDAC69E08pLYxyFRcLgn,Back to Stage,spotify:album:6SmAKP3u529E7LpnjFsSVf,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,10,242320


In [73]:
#### STEP 3. REMOVE LIVE/INSTRUMENTAL/ALTERNATIVE LANGUAGE VERSION OF SONGS ####
#### Logic: these are just repeats of the original version of the song. 

#### remove songs with the following words in the title: 
words_rm = [' \-live ', ' \-\slive ', '\(live\s',
            'instrumental', ' inst\. ', '\s inst \s ', '\(inst\)',
           'jp ver', 'japanese ver', 'eng ver', 'english ver',
            'korean ver', 'kr ver', 'karaoke']


### isolate song names
print('total song names: ', len(songs_clean['song_name']))
song_names = songs_clean['song_name'].drop_duplicates()
print('total number of unique song_names: ',len(song_names))
song_names.head(n=10)

### Filter for albums with the keywords
# songs_rm = songs_clean[songs_clean['song_name'].str.lower().str.contains('|'.join(words_rm))]['song_name'].drop_duplicates()
# print('total number of songs to remove: ',len(songs_rm))
# songs_rm

### removing songs with these keywords from the dataset
songs_clean = songs_clean[~songs_clean['song_name'].str.lower().str.contains('|'.join(words_rm))]
print('number of songs when instrumental versions are removed: ' , len(songs_clean))
songs_clean.head(n=25)

total song names:  157
total number of unique song_names:  108


0                                Delicious (Your Lips)
1                                                 뒹굴뒹굴
2                                     살아있네 Still Alive
3                                          Still Alive
4                                               Rewind
5                        방문을 닫으면 (Behind closed doors)
6                                               위험한 장난
7    Fire (feat. Conan O'Brien, Steven Yeun, 박지민 Ji...
8                                   엇갈렸어 (Feat. T 윤미래)
9                                          Criss Cross
Name: song_name, dtype: object

number of songs when instrumental versions are removed:  155


,song_name,song_uri,album,album_uri,artist,artist_uri,release_date,popularity,duration_ms
0,Delicious (Your Lips),spotify:track:5CkAYc5Y3leBLK6Wd59It0,Back to Stage,spotify:album:6SmAKP3u529E7LpnjFsSVf,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,12,214786
1,뒹굴뒹굴,spotify:track:0otd8yQz0lUg2lAVKqSWNl,Halftime,spotify:album:5p7fn6NyIiI1fhkh9mQlT0,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2013-09-09,0,203719
2,살아있네 Still Alive,spotify:track:4ERsViP3OTdJ9qIwpsUSTj,Still Alive,spotify:album:5HiT4lbTgn7VKDAX5Lecql,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2016-04-10,23,174953
3,Still Alive,spotify:track:2uDbAd6taMMeoyOAc9leBU,J.Y. Park BEST (Selected Edition),spotify:album:38rFlt6IxeZXsxPb7PcMn5,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2020-10-07,26,177733
4,Rewind,spotify:track:6fHcZvUm9G76hlLG3PjAfL,Sad Freedom,spotify:album:2OCK4oigskg6ZQ9mBm9zii,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2009-12-01,5,235441
5,방문을 닫으면 (Behind closed doors),spotify:track:6dmNuJysRUqX2CDA4q2MGs,24/34,spotify:album:7cPUxsJL1nMh87bld2LyTo,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2015-04-12,14,215955
6,위험한 장난,spotify:track:0mnP50PPZLrZm7tOTmui7N,Back To Stage,spotify:album:0R52ET7vG1Hy7eGouTxCve,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,5,199613
7,"Fire (feat. Conan O'Brien, Steven Yeun, 박지민 Ji...",spotify:track:0udhVlaFTeXvlmYWaFo66D,Still Alive,spotify:album:4ueWIlgAiwx4JyAHBCXOwt,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2016-04-10,32,185886
8,엇갈렸어 (Feat. T 윤미래),spotify:track:40nGrvrNoQlspovlWh8Acq,Back To Stage,spotify:album:41h8n1l24t2dq0N2e5lsll,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,1,242320
9,Criss Cross,spotify:track:1kXDAC69E08pLYxyFRcLgn,Back to Stage,spotify:album:6SmAKP3u529E7LpnjFsSVf,J.Y. Park,spotify:artist:1TTx0YcbKUtJIZY1HEnh9B,2007-11-16,10,242320


In [74]:
#### STEP 4: REMOVE DUPLICATE SONGS THAT HAVE DIFFERENT URIS #### 
#### songs are considered duplicates if the song name, artist name and album name match simultaneously.
#### songs of the same name and artist released on a different album may have been re-arranged, remixed etc.
#### These have been checked and do return the same audio features despite different song uris.
songs_clean = songs_clean.sort_values('popularity', ascending = False).drop_duplicates(subset = ['song_name', 'artist', 'album'], keep='first')
print('total songs: ', len(songs_clean))

total songs:  112


In [75]:
#### STEP 5: EXPORT ####
### Export clean song metadata which has undergone the following filterings:
#### 1. remove songs by artists who's URIs are not from the artist dataframe with the exception of songs
####    who had a relevant artist's name in the song or artist (some list multiple)
#### 2. remove live/tour/compilation albums
#### 3. remove live/instrumental/karaoke songs
#### 4. remove duplicates
songs_clean.to_csv('songs_meta_table_clean3.csv')

In [76]:
twinsongs = songs_clean[songs_clean.duplicated(subset = ['song_name', 'artist', 'album'], keep = False)].sort_values(by=['song_name'], ascending=True)
len(twinsongs)
twinsongs.head(n=10)

0

,song_name,song_uri,album,album_uri,artist,artist_uri,release_date,popularity,duration_ms


In [77]:
#### dropping duplicates
print('total unique songs: ', len(twinsongs.drop_duplicates(subset = ['song_name', 'artist', 'album'])))
dup_rm = twinsongs.sort_values('popularity', ascending = False).drop_duplicates(subset = ['song_name', 'artist', 'album'], keep='first')
dup_rm.sort_values(by=['song_name'], ascending=True).head(n=10)
print('duplicates removed keeping most popular', len(dup_rm))

total unique songs:  0


,song_name,song_uri,album,album_uri,artist,artist_uri,release_date,popularity,duration_ms


duplicates removed keeping most popular 0
